In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import time

import torch

In [2]:
torch.device("mps" if torch.backends.mps.is_available() else "cpu")

device(type='mps')

Primero tenemos que sacar la matriz de diseño:

$$
\begin{bmatrix}
1 & x_{12} & \cdots & x_{1p} \\
1 & x_{22} & \cdots & x_{2p} \\
\vdots & \vdots & \ddots & \vdots \\
1 & x_{n2} & \cdots & x_{np} \\
\end{bmatrix}
$$

$ y = \text{Pago_i}, \text{i=0, ..., n} $,

$ y_i \mid \beta, \sigma^2 \sim \mathcal{N}(\mu, \sigma^2) $, --> Likelihood,


$ \beta \mid \mathcal{N}_{p+1}(\mu, \sigma^2)$,

El resto esta en el freeform...

# PP

In [7]:
PP_ABC = pd.read_csv('PP_Segmentado_ABC.csv')

In [8]:
PP_ABC

,Socio,Mob,Saldo_total,Saldo_Mes,Pago_minimo_M0,Utilizacion,Pago_M0,Limite_credito,Genero,ESTADO,...,Edad,Variable_Objetivo_m6,Variable_Objetivo_m5,Variable_Objetivo_m4,Variable_Objetivo_m3,Variable_Objetivo_m2,Variable_Objetivo_m1,L2_Saldo_Mes,L2_Pago,Segmento_ABC
0,BOD,4,68086.86,68086.86,68086.86,15.130413,NaN,4500.0,M,EM,...,56,0,0,0,0,0,0,73701.086634,0.0,A
1,BOD,15,39558.87,39558.87,39558.87,15.823548,NaN,2500.0,M,GRO,...,62,0,0,0,0,0,0,67215.740776,0.0,A
2,GCC,17,36364.76,36364.76,36364.76,3.827869,NaN,9500.0,F,GRO,...,49,0,0,0,0,0,0,62592.930267,0.0,A
3,GCC,17,32796.07,32796.07,32796.07,3.999521,NaN,8200.0,F,GRO,...,59,0,0,0,0,0,0,55957.640182,0.0,A
4,BOD,14,34526.00,33490.35,33490.35,7.192917,NaN,4800.0,M,GRO,...,62,0,0,0,0,0,0,55222.334312,0.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126110,BOD,2,691.01,202.63,202.63,0.072738,692.0,9500.0,F,DF,...,38,0,0,0,0,0,0,229.211467,0.0,C
126111,CYA,62,119.01,119.01,119.01,0.019478,NaN,6110.0,F,GRO,...,30,0,0,0,0,0,0,170.517831,0.0,C
126112,CYA,34,124.61,124.61,124.61,0.062305,125.0,2000.0,M,DF,...,50,0,0,0,0,0,0,124.610000,0.0,C
126113,GCC,9,68.31,68.31,68.31,0.013394,NaN,5100.0,M,GTO,...,31,0,0,0,0,0,0,121.618722,0.0,C


Observando los datos de primera vista hay muchos meses en pago = 0, al igual al sacar la norma va a salir muy bajo su norma.

In [9]:
from sklearn.model_selection import train_test_split

X = PP_ABC.drop('Segmento_ABC', axis=1)  # features
y = PP_ABC['Segmento_ABC']               # variable estratificadora

X_subset, _, y_subset, _ = train_test_split(
    X, y,
    test_size=0.9,      # mantener 30%
    stratify=y,         # estratificar por PP_ABC
    random_state=42
)

In [10]:
PP_ABC = pd.concat([X_subset, y_subset], axis=1)
PP_ABC.to_csv('PP_ss.csv', index=False)
PP_ABC

,Socio,Mob,Saldo_total,Saldo_Mes,Pago_minimo_M0,Utilizacion,Pago_M0,Limite_credito,Genero,ESTADO,...,Edad,Variable_Objetivo_m6,Variable_Objetivo_m5,Variable_Objetivo_m4,Variable_Objetivo_m3,Variable_Objetivo_m2,Variable_Objetivo_m1,L2_Saldo_Mes,L2_Pago,Segmento_ABC
63041,CYA,21,5073.81,3738.97,3738.97,0.929269,NaN,5460.0,M,PUE,...,26,0,0,0,0,0,0,4950.653594,2340.656745,A
39059,PRO,38,6099.60,2999.53,2999.53,1.196000,NaN,5100.0,M,EM,...,41,0,0,0,0,0,0,6711.104274,3210.124608,A
94015,CYA,33,5798.35,1502.64,1502.64,1.136931,2000.0,5100.0,F,COA,...,64,0,0,0,0,0,0,3438.247142,2615.893729,B
53167,CYA,51,4990.62,3051.82,3051.82,1.279646,NaN,3900.0,F,JAL,...,36,0,0,0,0,0,0,5531.346250,2619.116263,A
19387,GCC,5,8676.52,8676.52,8676.52,1.701278,NaN,5100.0,F,BCN,...,32,0,0,0,0,0,0,9802.365064,1095.692019,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110721,CYA,4,3115.97,1744.41,1744.41,0.277468,NaN,11230.0,F,VER,...,40,0,0,0,0,0,0,2657.077671,1691.511104,C
61833,CYA,18,7181.28,2879.87,2879.87,1.408094,NaN,5100.0,F,EM,...,33,0,0,0,0,0,0,5016.212370,3799.381397,A
2195,SHA,12,14732.38,14732.38,14732.38,3.101554,NaN,4750.0,F,DF,...,25,0,0,0,0,0,0,17331.531145,2480.875853,A
3050,CYA,33,13412.45,13412.45,13412.45,3.548267,NaN,3780.0,M,EM,...,35,0,0,0,0,0,0,16254.023609,1625.000000,A


## get_dummies para Segmento_ABC

In [49]:
# one hot encode del segmento ABC
PP_dummy = pd.get_dummies(PP_ABC['Segmento_ABC'], drop_first=True, dtype=int)

In [50]:
# PP_dummy

## Instanciamos las variables del modelo!!

In [51]:
###################### PROBAMOS CON DIFERENTES VARIABLE PARA EL MODELO LINEAL ######################

######## SOLO USAMOS LOS SEGMENTOS ABC ########
# x = np.column_stack((np.ones(len(PP_dummy)), PP_dummy['B'], PP_dummy['C']))  

######## SOLO USAMOS LOS SEGMENTOS ABC + NORMA SALDO MES ########
# x = np.column_stack((np.ones(len(PP_dummy)), PP_dummy['B'], PP_dummy['C'], PP_ABC['L2_Saldo_Mes']))

######## SOLO USAMOS LOS SEGMENTOS ABC + VARIABLE_OBJETIVO_Mi ########
# x = np.column_stack((np.ones(len(PP_dummy)), PP_dummy['B'], PP_dummy['C'], PP_ABC['Variable_Objetivo_m6'], PP_ABC['Variable_Objetivo_m5'], PP_ABC['Variable_Objetivo_m4'], PP_ABC['Variable_Objetivo_m3'], PP_ABC['Variable_Objetivo_m2'], PP_ABC['Variable_Objetivo_m1']))


In [52]:
x = np.column_stack((np.ones(len(PP_dummy)), PP_dummy['B'], PP_dummy['C'], PP_ABC['dias_deudados_m2']))

Y = np.array(PP_ABC['Variable_objetivo'])


## Prior & Posterior Dist.

In [53]:
############### NUMERO DE SIMULACIONES
S = 10000
################ PARAMETROS DE LIKELIHOOD DEL MODELO
beta = np.zeros((S + 1, x.shape[1])) # 4 es x que son columnas
sigma = np.ones((S + 1,1)) # un vector de sigmas

################ inicializca parametros de NORMAL prior para BETA
beta_0 = np.zeros((x.shape[1]))
SIGMA_0 = 100 * np.eye(len(beta_0))
inv_SIGMA_0 = np.linalg.inv(SIGMA_0)

################ HIPERPARAMETROS DE LA GAMMA PRIOR PARA 1/sigma
nu_0 = 2*0.001 # este hiperparametro 0.001 son valores que se eligen para que la prior no sea informativa. alpha tiene que ser 0.001 para la dist. gamma
sigma2_0 = (2*0.001)/nu_0 # igual este 0.001 se elige para el valor de beta de la dist. gamma

################ necesario para calculo
n = len(x)

prob = np.ones((S + 1,x.shape[0])) # 4 es x que son columnas

################ Instanciamos estas operaciones por fuera del for para optimizar el ciclado ###############
xt_X_x = x.T @ x
xt_X_Y = x.T @ Y
inv_SIGMA_0_X_beta_0 = inv_SIGMA_0 @ beta_0

alpha_post_gamma_inv = (nu_0 + n)/2 # Parametro Alpha de la gamma inversa de sigma^2 de posterior
nu_0_X_sigma2_0 = nu_0*sigma2_0

############################## POSTERIOR FOR BETA AND SIGMA ##############################
start_time = time.time()

for s in range(S):
    current_time = time.time()

    # primero se calcula el vector de medias de x - meu
    # luego se calcula la matriz de covarianza  - sigma

    var = np.linalg.inv(inv_SIGMA_0 + (xt_X_x / sigma[s])) 
    mu = var @ (inv_SIGMA_0_X_beta_0 + (xt_X_Y / sigma[s]))
    
    beta[s+1, :] = np.random.multivariate_normal(mu, var, size=1)
    sigma[s+1] = 1/np.random.gamma(alpha_post_gamma_inv, 1/(nu_0_X_sigma2_0 + 0.5*(Y - x @ beta[s + 1, :]).T @ (Y - x @ beta[s + 1, :])), size=1)
    
    if s == 0:
        print(f"First iteration: {current_time - start_time:.6f} seconds")
        last_time = current_time
        print("-" * 30)
    elif s % 100 == 0:
        print(f"{s} - Time since last iteration: {current_time - last_time:.6f} seconds")
        last_time = current_time
        print("-" * 30)
    elif s == S - 1:
        print(f"Total time: {last_time - start_time:.6f} seconds")  

First iteration: 0.000184 seconds
------------------------------
100 - Time since last iteration: 1.836682 seconds
------------------------------
200 - Time since last iteration: 0.799813 seconds
------------------------------
300 - Time since last iteration: 0.626045 seconds
------------------------------
400 - Time since last iteration: 1.094205 seconds
------------------------------
500 - Time since last iteration: 2.886985 seconds
------------------------------
600 - Time since last iteration: 0.942228 seconds
------------------------------
700 - Time since last iteration: 0.115970 seconds
------------------------------
800 - Time since last iteration: 0.075973 seconds
------------------------------
900 - Time since last iteration: 0.141217 seconds
------------------------------
1000 - Time since last iteration: 0.119202 seconds
------------------------------
1100 - Time since last iteration: 0.101719 seconds
------------------------------
1200 - Time since last iteration: 0.165242

Aqui ya obtenemos la beta y sigma para cada sujeto:

In [54]:
beta

array([[0.        , 0.        , 0.        , 0.        ],
       [0.23623951, 0.11804771, 0.22369479, 0.00162993],
       [0.23228757, 0.12342052, 0.22898882, 0.00176734],
       ...,
       [0.24136602, 0.11680774, 0.22727258, 0.00177936],
       [0.23786073, 0.11912751, 0.22807083, 0.00173213],
       [0.24068073, 0.1138051 , 0.23764088, 0.00182437]])

In [55]:
sigma

array([[1.        ],
       [0.17287715],
       [0.17107997],
       ...,
       [0.17352177],
       [0.1709276 ],
       [0.17157775]])

## Probability Thresholds $\mathbb{P} (y \ge y_0)$

In [56]:
prob_test = np.ones((n, S + 1)) # Una matriz que guardara las probs
post_mean = x @ beta.T # Esto es una funcion de x (no necesariamente la posterior mean)

# Esto es un threshold que determina si un sujeto i va a pagar o no dado ese threshold
    
# for sujeto in range(n):
#     prob[s, sujeto] = 1 - stats.norm.cdf(y_0, x[sujeto] @ beta[s+1, :], np.sqrt(sigma[s+1]))


######################## DISTINTOS THRESHOLDS PARA PROBAR SI PAGO >= Y_0 ########################
# y_0 = np.where(PP_ABC['Segmento_ABC'] == 'A', PP_ABC.loc[PP_ABC['Segmento_ABC'] == 'A', 'L2_Pago'].mean(),
#           np.where(PP_ABC['Segmento_ABC'] == 'B', PP_ABC.loc[PP_ABC['Segmento_ABC'] == 'B', 'L2_Pago'].mean(), 
#           np.where(PP_ABC['Segmento_ABC'] == 'C', PP_ABC.loc[PP_ABC['Segmento_ABC'] == 'C', 'L2_Pago'].mean(), 
#                    0)))

y_0 = 0

# y_0 = np.array(PP_ABC['Pago_minimo_M0'])

######################## AHORA SI CALCULAMOS LA PROBABILIDAD PARA TODOS LOS SUJETOS ########################

start_time = time.time()
for i in range(n):
    current_time = time.time()

    # prob_test[i, :] = 1 - stats.norm.cdf(y_0[i], post_mean[i, :], np.sqrt(sigma).flatten()) # For y_0 as a vector
    prob_test[i, :] = 1 - stats.norm.cdf(y_0, post_mean[i, :], np.sqrt(sigma).flatten())

    
    if i == 0:
        print(f"First iteration: {current_time - start_time:.6f} seconds")
        last_time = current_time
        print("-" * 30)

    elif i % 1000 == 0:
        print(f"{i} - Time since last iteration: {current_time - last_time:.6f} seconds")
        last_time = current_time
        print("-" * 30)
    elif i == n - 1:
        print(f"Total time: {last_time - start_time:.6f} seconds")  

First iteration: 0.003975 seconds
------------------------------
1000 - Time since last iteration: 0.907849 seconds
------------------------------
2000 - Time since last iteration: 0.414020 seconds
------------------------------
3000 - Time since last iteration: 0.408447 seconds
------------------------------
4000 - Time since last iteration: 0.401021 seconds
------------------------------
5000 - Time since last iteration: 0.376313 seconds
------------------------------
6000 - Time since last iteration: 0.588014 seconds
------------------------------
7000 - Time since last iteration: 0.410996 seconds
------------------------------
8000 - Time since last iteration: 0.392837 seconds
------------------------------
9000 - Time since last iteration: 0.381779 seconds
------------------------------
10000 - Time since last iteration: 0.374316 seconds
------------------------------
11000 - Time since last iteration: 0.323717 seconds
------------------------------
12000 - Time since last iterati

In [57]:
# np.unique(y_0, return_counts=True) # para el y_0 de np.mean() para cada segmento

In [58]:
post_mean[:5, -100:]

array([[0.07605436, 0.07716452, 0.0771915 , 0.07444334, 0.07411772,
        0.07901973, 0.07299749, 0.08008076, 0.07580979, 0.07565557,
        0.07415242, 0.07881743, 0.07852435, 0.07558692, 0.07419915,
        0.072727  , 0.069469  , 0.07491792, 0.08092023, 0.08026614,
        0.08127668, 0.07340932, 0.07346227, 0.07486906, 0.07168573,
        0.06903946, 0.0716015 , 0.07503214, 0.07692372, 0.07993357,
        0.07807776, 0.07783384, 0.07702241, 0.08087113, 0.07412403,
        0.07340235, 0.07289463, 0.07279446, 0.07083923, 0.07449743,
        0.07690283, 0.07426783, 0.08053847, 0.07052058, 0.07488428,
        0.07506381, 0.07687126, 0.07330604, 0.07391215, 0.07426793,
        0.070523  , 0.07663   , 0.0726748 , 0.07728407, 0.07084459,
        0.07827443, 0.07817941, 0.07535935, 0.07227259, 0.07463728,
        0.07455735, 0.07524634, 0.07453499, 0.07583804, 0.07307246,
        0.07761529, 0.07922899, 0.07406365, 0.07585546, 0.07665759,
        0.07216807, 0.07567752, 0.08094261, 0.07

In [59]:
# Segmento A
prob_test[:5, -100:]

array([[0.57289612, 0.57364722, 0.57418455, 0.571255  , 0.57106556,
        0.5755602 , 0.57021922, 0.57646808, 0.57276699, 0.57215874,
        0.57112509, 0.57567595, 0.57520427, 0.57241081, 0.57108042,
        0.56960932, 0.56635152, 0.57191164, 0.57731181, 0.57676283,
        0.57775605, 0.57035908, 0.57072035, 0.57178399, 0.56849818,
        0.56599306, 0.56840589, 0.57191292, 0.57405814, 0.57683778,
        0.57469508, 0.57391502, 0.57370109, 0.57740457, 0.5709521 ,
        0.57021095, 0.56982745, 0.57000962, 0.56764845, 0.57123837,
        0.57352204, 0.57117631, 0.57708044, 0.56715756, 0.57205371,
        0.57218288, 0.57380101, 0.57023556, 0.57103774, 0.57127399,
        0.56744725, 0.57371381, 0.56946533, 0.57422241, 0.56779521,
        0.57472657, 0.57489938, 0.571852  , 0.56908362, 0.57147658,
        0.57163183, 0.57219372, 0.57105884, 0.57253085, 0.5702336 ,
        0.57410592, 0.57580747, 0.57098647, 0.57291105, 0.57304328,
        0.56909736, 0.57218599, 0.57752416, 0.56

In [60]:
# Segmento B
prob_test[76130:76135, -100:]

array([], shape=(0, 100), dtype=float64)

In [61]:
# Segmento C
prob_test[76125 + 32425:76131 + 32425 + 5, -100:]

array([], shape=(0, 100), dtype=float64)

In [62]:
post_mean.shape

(63057, 10001)

In [63]:
prob_test.shape

(63057, 10001)